## Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

## Import Data

In [2]:
df = pd.read_csv('Data/loan.csv')

/tmp/ipykernel_2413/1715284255.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/loan.csv')


In [3]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [4]:
df.info(verbose=True, null_counts=True)

/tmp/ipykernel_2413/2612161665.py:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df.info(verbose=True, null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 111 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    id                              39717 non-null  int64  
 1    member_id                       39717 non-null  int64  
 2    loan_amnt                       39717 non-null  int64  
 3    funded_amnt                     39717 non-null  int64  
 4    funded_amnt_inv                 39717 non-null  float64
 5    term                            39717 non-null  object 
 6    int_rate                        39717 non-null  object 
 7    installment                     39717 non-null  float64
 8    grade                           39717 non-null  object 
 9    sub_grade                       39717 non-null  object 
 10   emp_title                       37258 non-null  object 
 11   emp_length                      38642 non-null  object 
 12   home_ownership  

## Drop all null columns

In [5]:
df.dropna(axis=1, how='all', inplace=True)

## Below listed are columns which are already clean

In [6]:
df['last_pymnt_amnt'].value_counts()

0.00       74
276.06     21
200.00     17
50.00      16
100.00     15
           ..
1763.87     1
172.27      1
889.67      1
150.73      1
256.59      1
Name: last_pymnt_amnt, Length: 34930, dtype: int64

In [7]:
df['total_pymnt_inv'].value_counts()

0.00        165
6514.52      16
5478.39      14
13148.14     14
11196.57     12
           ... 
17702.50      1
19026.06      1
7355.24       1
387.55        1
980.83        1
Name: total_pymnt_inv, Length: 37518, dtype: int64

In [8]:
df['total_rec_prncp'].value_counts()

10000.00    2293
12000.00    1805
5000.00     1702
6000.00     1637
15000.00    1400
            ... 
1097.81        1
1410.30        1
6968.65        1
3477.49        1
16077.42       1
Name: total_rec_prncp, Length: 7976, dtype: int64

In [9]:
df['total_rec_int'].value_counts()

0.00       71
1196.57    26
514.52     19
956.78     17
1784.23    17
           ..
494.53      1
1119.88     1
62.31       1
2656.10     1
1695.26     1
Name: total_rec_int, Length: 35148, dtype: int64

In [10]:
df['total_rec_late_fee'].value_counts()

0.000000     37671
15.000000      255
15.000000       58
30.000000       55
15.000000       47
             ...  
35.286832        1
15.000000        1
14.777500        1
14.967774        1
19.890000        1
Name: total_rec_late_fee, Length: 1356, dtype: int64

In [11]:
df['recoveries'].value_counts()

0.00       35499
11.29          4
10.40          4
10.66          3
44.92          3
           ...  
764.69         1
653.08         1
1080.96        1
878.19         1
21.29          1
Name: recoveries, Length: 4040, dtype: int64

In [12]:
df['collection_recovery_fee'].value_counts()

0.0000      35935
2.0000         12
1.2000         10
3.7100          9
1.8800          8
            ...  
3.7900          1
773.4900        1
272.8250        1
1.7697          1
0.2300          1
Name: collection_recovery_fee, Length: 2616, dtype: int64

In [13]:
df['purpose'].value_counts()

debt_consolidation    18641
credit_card            5130
other                  3993
home_improvement       2976
major_purchase         2187
small_business         1828
car                    1549
wedding                 947
medical                 693
moving                  583
vacation                381
house                   381
educational             325
renewable_energy        103
Name: purpose, dtype: int64

In [14]:
df['zip_code'].value_counts()

100xx    597
945xx    545
112xx    516
606xx    503
070xx    473
        ... 
381xx      1
378xx      1
739xx      1
396xx      1
469xx      1
Name: zip_code, Length: 823, dtype: int64

In [15]:
df['addr_state'].value_counts()

CA    7099
NY    3812
FL    2866
TX    2727
NJ    1850
IL    1525
PA    1517
VA    1407
GA    1398
MA    1340
OH    1223
MD    1049
AZ     879
WA     840
CO     792
NC     788
CT     751
MI     720
MO     686
MN     615
NV     497
SC     472
WI     460
AL     452
OR     451
LA     436
KY     325
OK     299
KS     271
UT     258
AR     245
DC     214
RI     198
NM     189
WV     177
HI     174
NH     171
DE     114
MT      85
WY      83
AK      80
SD      64
VT      54
MS      19
TN      17
IN       9
ID       6
IA       5
NE       5
ME       3
Name: addr_state, dtype: int64

In [16]:
df['dti'].value_counts()

0.00     183
12.00     51
18.00     45
19.20     40
13.20     39
        ... 
29.13      1
25.31      1
29.76      1
28.42      1
25.43      1
Name: dti, Length: 2868, dtype: int64

In [17]:
df['delinq_2yrs'].value_counts()

0     35405
1      3303
2       687
3       220
4        62
5        22
6        10
7         4
8         2
9         1
11        1
Name: delinq_2yrs, dtype: int64

In [18]:
df['inq_last_6mths'].value_counts()

0    19300
1    10971
2     5812
3     3048
4      326
5      146
6       64
7       35
8       15
Name: inq_last_6mths, dtype: int64

In [19]:
df['open_acc'].value_counts()

7     4018
6     3946
8     3936
9     3718
10    3223
5     3183
11    2746
4     2343
12    2273
13    1911
3     1493
14    1487
15    1177
16     940
17     741
2      605
18     533
19     396
20     289
21     244
22     143
23      97
24      81
25      55
26      34
28      25
27      22
30      15
29      13
31       7
34       5
32       4
35       4
33       3
36       2
39       1
38       1
44       1
41       1
42       1
Name: open_acc, dtype: int64

In [20]:
df['pub_rec'].value_counts()

0    37601
1     2056
2       51
3        7
4        2
Name: pub_rec, dtype: int64

In [21]:
df['revol_bal'].value_counts()

0        994
298       14
255       14
1         12
682       11
        ... 
21424      1
30747      1
23862      1
20197      1
85607      1
Name: revol_bal, Length: 21711, dtype: int64

In [22]:
df['total_acc'].value_counts()

16    1471
15    1462
17    1457
14    1445
20    1428
      ... 
74       1
77       1
78       1
87       1
90       1
Name: total_acc, Length: 82, dtype: int64

In [23]:
df['out_prncp'].value_counts()

0.00       38577
1972.60        2
827.13         2
2277.11        2
2963.24        2
           ...  
782.23         1
2296.41        1
1928.85        1
1061.32        1
79.24          1
Name: out_prncp, Length: 1137, dtype: int64

In [24]:
df['out_prncp_inv'].value_counts()

0.00       38577
1972.60        2
1664.64        2
827.13         2
1863.21        1
           ...  
782.23         1
2289.14        1
1928.85        1
1061.32        1
79.24          1
Name: out_prncp_inv, Length: 1138, dtype: int64

In [25]:
df['installment'].value_counts()

311.11     68
180.96     59
311.02     54
150.80     48
368.45     46
           ..
1224.46     1
63.44       1
157.67      1
492.34      1
255.43      1
Name: installment, Length: 15383, dtype: int64

In [26]:
df['verification_status'].value_counts()

Not Verified       16921
Verified           12809
Source Verified     9987
Name: verification_status, dtype: int64

In [27]:
df['loan_status'].value_counts()

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

## These columns are redundant and are dropped

In [28]:
df['policy_code'].value_counts()

1    39717
Name: policy_code, dtype: int64

In [29]:
df['application_type'].value_counts()

INDIVIDUAL    39717
Name: application_type, dtype: int64

In [30]:
df['acc_now_delinq'].value_counts()

0    39717
Name: acc_now_delinq, dtype: int64

In [31]:
df['delinq_amnt'].value_counts()

0    39717
Name: delinq_amnt, dtype: int64

In [32]:
df['initial_list_status'].value_counts()

f    39717
Name: initial_list_status, dtype: int64

In [33]:
df['pymnt_plan'].value_counts()

n    39717
Name: pymnt_plan, dtype: int64

In [34]:
df['collections_12_mths_ex_med'].value_counts(dropna=False)

0.0    39661
NaN       56
Name: collections_12_mths_ex_med, dtype: int64

In [35]:
df['chargeoff_within_12_mths'].value_counts(dropna=False)

0.0    39661
NaN       56
Name: chargeoff_within_12_mths, dtype: int64

In [36]:
df['tax_liens'].value_counts(dropna=False)

0.0    39678
NaN       39
Name: tax_liens, dtype: int64

In [37]:
df['next_pymnt_d'].value_counts(dropna=False)

NaN       38577
Jun-16     1125
Jul-16       15
Name: next_pymnt_d, dtype: int64

In [38]:
df.drop(['collections_12_mths_ex_med','chargeoff_within_12_mths','tax_liens','id','member_id','next_pymnt_d'], axis=1, inplace=True)
df.drop('pymnt_plan', axis=1, inplace=True)
df.drop('initial_list_status', axis=1, inplace=True)
df.drop('policy_code', axis=1, inplace=True)
df.drop('application_type', axis=1, inplace=True)
df.drop('acc_now_delinq', axis=1, inplace=True)
df.drop('delinq_amnt', axis=1, inplace=True)

## Cleaning percentages

In [39]:
df['int_rate']

0        10.65%
1        15.27%
2        15.96%
3        13.49%
4        12.69%
          ...  
39712     8.07%
39713    10.28%
39714     8.07%
39715     7.43%
39716    13.75%
Name: int_rate, Length: 39717, dtype: object

In [40]:
def clean_percentages(i):
    i = i.str.replace('%', '')
    i = i.astype('float64')
    i = i/100
    return i

In [41]:
df['int_rate'] = clean_percentages(df['int_rate'])

## Figure out what to do with the below columns

In [42]:
df['pub_rec_bankruptcies'].value_counts(dropna=False)
#figure out what to do with the null values

0.0    37339
1.0     1674
NaN      697
2.0        7
Name: pub_rec_bankruptcies, dtype: int64

In [43]:
df['emp_length'].value_counts(dropna=False)
# Figure out what to do with the null values, maybe nan values refers to 0 years of work experience?

10+ years    8879
< 1 year     4583
2 years      4388
3 years      4095
4 years      3436
5 years      3282
1 year       3240
6 years      2229
7 years      1773
8 years      1479
9 years      1258
NaN          1075
Name: emp_length, dtype: int64

In [44]:
df['mths_since_last_delinq'].value_counts(dropna=False)
#Can drop this since most are missing values? Investigate further

NaN      25682
0.0        443
15.0       252
30.0       247
23.0       247
         ...  
89.0         1
107.0        1
85.0         1
97.0         1
95.0         1
Name: mths_since_last_delinq, Length: 96, dtype: int64

In [45]:
df['mths_since_last_record'].value_counts(dropna=False)
# can drop this column since most of the values are missing? Investigate further

NaN      36931
0.0        670
104.0       61
89.0        60
113.0       59
         ...  
19.0         1
20.0         1
120.0        1
129.0        1
5.0          1
Name: mths_since_last_record, Length: 112, dtype: int64

In [46]:
df['revol_util'].value_counts(dropna=False)

# figure out what to do with the missing values.
# How is this dtype int64 even though percentage symbol is present in this column. Clean the percentage symbol

0%        977
0.20%      63
63%        62
0.10%      58
66.70%     58
         ... 
77.63%      1
25.74%      1
62.31%      1
0.83%       1
7.28%       1
Name: revol_util, Length: 1090, dtype: int64

In [47]:
df['emp_title'].value_counts(dropna=False)
# can we replace nan values with 'NA'?

NaN                                2459
US Army                             134
Bank of America                     109
IBM                                  66
AT&T                                 59
                                   ... 
AMEC                                  1
lee county sheriff                    1
Bacon County Board of Education       1
Hotpads                               1
Evergreen Center                      1
Name: emp_title, Length: 28821, dtype: int64

In [48]:
df['title'].value_counts(dropna=False)
# can we replace nan values with 'NA'?

Debt Consolidation                  2184
Debt Consolidation Loan             1729
Personal Loan                        659
Consolidation                        517
debt consolidation                   505
                                    ... 
your rate is better than my rate       1
Concession Trailer                     1
gregs                                  1
EZover                                 1
JAL Loan                               1
Name: title, Length: 19616, dtype: int64

In [49]:
df['term'].unique()
# Is it necessary to convert this categorical column into numeric format for analysis?

array([' 36 months', ' 60 months'], dtype=object)

In [50]:
df.url
# we can drop this column? Are there any insights that can be derived from this column?
# df.drop('url', axis=1, inplace=True)

0        https://lendingclub.com/browse/loanDetail.acti...
1        https://lendingclub.com/browse/loanDetail.acti...
2        https://lendingclub.com/browse/loanDetail.acti...
3        https://lendingclub.com/browse/loanDetail.acti...
4        https://lendingclub.com/browse/loanDetail.acti...
                               ...                        
39712    https://lendingclub.com/browse/loanDetail.acti...
39713    https://lendingclub.com/browse/loanDetail.acti...
39714    https://lendingclub.com/browse/loanDetail.acti...
39715    https://lendingclub.com/browse/loanDetail.acti...
39716    https://lendingclub.com/browse/loanDetail.acti...
Name: url, Length: 39717, dtype: object

In [51]:
df.desc.value_counts(dropna=False)
# Since we are not doing NLP, we can drop this column?
# df.drop('desc', axis=1, inplace=True)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           12940
                                                                                                                                                                                                                                                                                                                                                    

In [52]:
df.total_pymnt.value_counts()
#why are there 26 same occurences of total payment?

11196.569430    26
0.000000        16
11784.232230    16
10956.775960    16
5478.387981     15
                ..
17768.430010     1
12794.806580     1
6193.803706      1
34797.769170     1
9195.263334      1
Name: total_pymnt, Length: 37850, dtype: int64

## Working with datetime values

In [53]:
def clean_date(i):
    i = i.str.replace('-', ' ')
    i = i.str[:4]+'20'+i.str[4:]
    i = pd.to_datetime(i)
    return(i)

In [54]:
df['issue_d'] = clean_date(df['issue_d'])

In [55]:
df['last_credit_pull_d'] = clean_date(df['last_credit_pull_d'])

In [56]:
df[df['last_credit_pull_d'].isna()]
# Remove these two rows?

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies
26025,5000,5000,5000.0,36 months,0.1112,163.98,B,B3,NaN,5 years,...,3362.96,2553.55,702.45,0.0,106.96,1.34,Jun-12,163.98,NaT,0.0
39476,1000,1000,875.0,36 months,0.0712,30.94,A,A1,NaN,< 1 year,...,971.96,1000.00,110.81,0.0,0.00,0.00,Aug-10,185.80,NaT,NaN


In [57]:
df['last_pymnt_d'] = clean_date(df['last_pymnt_d'])

In [58]:
df[df['last_pymnt_d'].isna()]
# Remove these 71 rows?

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies
26,15000,15000,8725.0,36 months,0.1427,514.64,C,C2,nyc transit,9 years,...,0.00,0.0,0.0,0.0,0.00,0.0000,NaT,0.0,2016-05-01,0.0
2724,8000,8000,3975.0,36 months,0.1465,275.96,C,C3,Southern Felt,10+ years,...,0.00,0.0,0.0,0.0,0.00,0.0000,NaT,0.0,2016-05-01,0.0
3613,6400,6400,6400.0,60 months,0.1825,163.39,D,D5,akrons children hospitial,5 years,...,366.75,0.0,0.0,0.0,366.75,66.0150,NaT,0.0,2012-04-01,0.0
3985,18000,18000,18000.0,60 months,0.1903,467.23,E,E2,ZIMFA INC,4 years,...,0.00,0.0,0.0,0.0,0.00,0.0000,NaT,0.0,2016-04-01,0.0
4206,1800,1800,1800.0,36 months,0.0890,57.16,A,A5,Hospira worldwide,9 years,...,1800.00,0.0,0.0,0.0,1800.00,0.0000,NaT,0.0,2016-05-01,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36185,20000,20000,13975.0,36 months,0.1189,663.28,B,B4,US Army,10+ years,...,3139.36,0.0,0.0,0.0,4493.30,1381.4454,NaT,0.0,2009-12-01,0.0
36206,7800,7800,7725.0,36 months,0.1095,255.17,B,B1,FLACS,< 1 year,...,0.00,0.0,0.0,0.0,0.00,0.0000,NaT,0.0,2016-05-01,0.0
36209,2000,2000,2000.0,36 months,0.1600,70.32,E,E2,Portland State Uniersity Distance Learning Center,< 1 year,...,89.99,0.0,0.0,0.0,89.99,0.9000,NaT,0.0,2009-12-01,0.0
36937,25000,25000,11625.0,36 months,0.1379,851.89,C,C5,ARES Corporation,1 year,...,0.00,0.0,0.0,0.0,0.00,0.0000,NaT,0.0,2009-06-01,0.0
